In [ ]:
# Author : Venugopal
# python version 3.12.0

Spatial Regression

In [26]:
from pysal.model import spreg
from pysal.lib import weights
from scipy import stats
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import osmnx as ox
sns.set(style="whitegrid")

In [27]:
fp = r"E:/test2/y20s04_plus_landscape.csv"
data = pd.read_csv(fp)
data.columns

Index(['md_mvm_id', 'lat', 'long', 'toc', 'year', 'month', 'day',
       'season_type', 'season', 'max_sample_depth', 'cpyll', 'ph', 'cond25',
       'alk_acid', 'ca', 'mg', 'na', 'k', 'so4', 'cl', 'f', 'nh4_n',
       'no2_no3_n', 'po4_p', 'tot_p', 'si', 'fe', 'oxy', 'water_temp', 'tot_n',
       'abs420', 'toc_ton', 'toc_top', 'altitud', 'aro_area', 'forest',
       'water', 'marshland', 'Hygge', 'openground', 'farmland', 'mountain',
       'urban'],
      dtype='object')

In [28]:

# Read OSM data - get administrative boundaries

# define the place query
query = {'country': 'Sweden'}

# get the boundaries of the place (add additional buffer around the query)
boundaries = ox.geocode_to_gdf(query, buffer_dist=5000)

# Let's check the boundaries on a map
boundaries.explore()


C:\Users\vijay\AppData\Local\Temp\ipykernel_12652\2186732279.py:7: UserWarning: The buffer_dist argument has been deprecated and will be removed in a future release. Buffer your results directly, if desired.
  boundaries = ox.geocode_to_gdf(query, buffer_dist=5000)


In [29]:
# Create a GeoDataFrame
data["geometry"] = gpd.points_from_xy(data["long"], data["lat"])
data = gpd.GeoDataFrame(data, crs="epsg:4326")

# Filter data geographically
data = gpd.sjoin(data, boundaries[["geometry"]])
data = data.reset_index(drop=True)

# Check the first rows
data.head()

,md_mvm_id,lat,long,toc,year,month,day,season_type,season,max_sample_depth,...,forest,water,marshland,Hygge,openground,farmland,mountain,urban,geometry,index_right
0,54,60.638793,15.445276,4.2,2020,10,12,1,4,0.5,...,73.469462,10.837264,3.676818,12.016456,0.000000,0.000000,0.0,0.000000,POINT (15.44528 60.63879),0
1,55,59.805629,17.903998,13.2,2020,10,26,1,4,0.5,...,78.878348,3.402606,3.824474,2.837472,8.661184,2.395915,0.0,0.000000,POINT (17.90400 59.80563),0
2,56,60.027726,15.662726,8.9,2020,10,21,1,4,0.5,...,75.968555,10.497775,3.313430,6.915198,2.147880,0.236199,0.0,0.920962,POINT (15.66273 60.02773),0
3,57,60.162144,15.733446,10.3,2020,10,21,1,4,0.5,...,71.366119,13.571100,4.610804,9.689102,0.762875,0.000000,0.0,0.000000,POINT (15.73345 60.16214),0
4,58,60.645048,13.626457,15.9,2020,10,13,1,4,0.5,...,69.157181,8.154980,19.268154,2.410018,1.009667,0.000000,0.0,0.000000,POINT (13.62646 60.64505),0


In [30]:
# Here the tooltip parameter specifies which attributes are shown when hovering on top of the points
# The vmax parameter specifies the maximum value for the colormap (here, all 1000 dollars and above are combined)
data.explore(column="toc", cmap="Reds", scheme="quantiles", tooltip=["toc"], tiles="CartoDB positron")

In [31]:
explanatory_vars = ['ph', 'mg', 'fe', 'tot_p', 'water_temp', 'cpyll', 'oxy','si', 'cond25', 'nh4_n','toc_ton', 'toc_top','altitud','forest']

In [32]:
all_model_attributes = ["toc"] + explanatory_vars
has_nans = False
for attr in all_model_attributes:
    if data[attr].hasnans:
        has_nans = True
print("Has missing values:", has_nans)

Has missing values: False


In [33]:
data = data.dropna(subset=all_model_attributes).copy()

In [22]:
w = weights.KNN.from_shapefile('E:/test2/shapefile.shp', k=8)
w.transform = 'R'
w

In [34]:
m1 = spreg.OLS(data[['toc']].values, data[explanatory_vars].values,name_y = 'toc', name_x = explanatory_vars)

In [35]:
print(m1.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :         toc                Number of Observations:         105
Mean dependent var  :     10.0629                Number of Variables   :          15
S.D. dependent var  :      6.1138                Degrees of Freedom    :          90
R-squared           :      0.8074
Adjusted R-squared  :      0.7774
Sum squared residual:     748.763                F-statistic           :     26.9464
Sigma-square        :       8.320                Prob(F-statistic)     :   3.333e-26
S.E. of regression  :       2.884                Log likelihood        :    -252.123
Sigma-square ML     :       7.131                Akaike info criterion :     534.246
S.E of regression ML:      2.6704                Schwarz criterion     :     574.055

-----------------------------------------------------------------------------

Geographical Weighted Regression

In [37]:
import libpysal as ps
from mgwr.gwr import GWR
import pandas as pd

data = r"E:/test2/y20s04_plus_landscape.csv"
df = pd.read_csv(data)
df = df.fillna(0)

coords = list(zip(df['long'], df['lat']))

y = np.array(df['toc']).reshape((-1,1))

ph = np.array(df['ph']).reshape((-1,1))
mg = np.array(df['mg']).reshape((-1,1))
tot_p = np.array(df['tot_p']).reshape((-1,1))
water_temp = np.array(df['water_temp']).reshape((-1,1))
cpyll = np.array(df['cpyll']).reshape((-1,1))
oxy = np.array(df['oxy']).reshape((-1,1))
si = np.array(df['si']).reshape((-1,1))
cond25 = np.array(df['cond25']).reshape((-1,1))
nh4_n = np.array(df['nh4_n']).reshape((-1,1))
toc_ton = np.array(df['toc_ton']).reshape((-1,1))
toc_top = np.array(df['toc_top']).reshape((-1,1))
altitud = np.array(df['altitud']).reshape((-1,1))
forest = np.array(df['forest']).reshape((-1,1))

X = np.hstack([ph,mg,tot_p,water_temp,cpyll,oxy,si,cond25,nh4_n,toc_ton,toc_top,altitud,forest])

model = GWR(coords, y, X, bw=90.000, fixed=False, kernel='bisquare')
results = model.fit()

In [38]:

print(results.params.shape)

(105, 14)


In [39]:
results.summary()

Model type                                                         Gaussian
Number of observations:                                                 105
Number of covariates:                                                    14

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                            781.295
Log-likelihood:                                                    -254.356
AIC:                                                                536.711
AICc:                                                               544.105
BIC:                                                                357.784
R2:                                                                   0.799
Adj. R2:                                                              0.770

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------

In [40]:
index = np.arange(len(y))
test = index[-10:]
X_test = X[test]
coords_test = np.array(coords)[test]
model = GWR(coords, y, X, bw=94, fixed=False, kernel='bisquare')
results = model.predict(coords_test, X_test)
print(results.params.shape)

(10, 14)


Multiscale Geographically Weighted Regression

In [42]:
import numpy as np
import pandas as pd
import libpysal as ps
from mgwr.gwr import MGWR
from mgwr.sel_bw import Sel_BW


data = r"E:/test2/y20s04_plus_landscape.csv"
df = pd.read_csv(data)
df = df.fillna(0)

coords = list(zip(df['long'], df['lat']))

y = np.array(df['toc']).reshape((-1,1))

ph = np.array(df['ph']).reshape((-1,1))
mg = np.array(df['mg']).reshape((-1,1))
tot_p = np.array(df['tot_p']).reshape((-1,1))
water_temp = np.array(df['water_temp']).reshape((-1,1))
cpyll = np.array(df['cpyll']).reshape((-1,1))
oxy = np.array(df['oxy']).reshape((-1,1))
si = np.array(df['si']).reshape((-1,1))
cond25 = np.array(df['cond25']).reshape((-1,1))
nh4_n = np.array(df['nh4_n']).reshape((-1,1))
toc_ton = np.array(df['toc_ton']).reshape((-1,1))
toc_top = np.array(df['toc_top']).reshape((-1,1))
altitud = np.array(df['altitud']).reshape((-1,1))
forest = np.array(df['forest']).reshape((-1,1))

X = np.hstack([ph,mg,tot_p,water_temp,cpyll,oxy,si,cond25,nh4_n,toc_ton,toc_top,altitud,forest])

X = (X - X.mean(axis=0)) / X.std(axis=0)
y = (y - y.mean(axis=0)) / y.std(axis=0)

selector = Sel_BW(coords, y, X, multi=True)
selector.search(multi_bw_min=[2])

Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

array([ 56.,  90.,  37., 104.,  66.,  99., 104.,  47.,  36., 104.,  51.,
        25., 104.,  49.])

In [43]:
model = MGWR(coords, y, X, selector, fixed=False, kernel='bisquare', sigma2_v1=True)
results = model.fit()

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

In [44]:
print(results.params.shape)

(105, 14)


In [45]:
results.summary()

Model type                                                         Gaussian
Number of observations:                                                 105
Number of covariates:                                                    14

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                             21.103
Log-likelihood:                                                     -64.751
AIC:                                                                157.502
AICc:                                                               164.895
BIC:                                                               -402.407
R2:                                                                   0.799
Adj. R2:                                                              0.770

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------

Spatial Lag Model (Wx)

In [6]:
from pysal.model import spreg
from pysal.lib import weights
from scipy import stats
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import osmnx as ox
sns.set(style="whitegrid")

c:\Users\vijay\.conda\envs\ox\Lib\site-packages\pysal\explore\segregation\network\network.py:15: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  warn(
c:\Users\vijay\.conda\envs\ox\Lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [7]:
fp1 = r"E:/test2/y20s04_plus_landscape.csv"
landdata = pd.read_csv(fp1)
landdata.columns

Index(['md_mvm_id', 'lat', 'long', 'toc', 'year', 'month', 'day',
       'season_type', 'season', 'max_sample_depth', 'cpyll', 'ph', 'cond25',
       'alk_acid', 'ca', 'mg', 'na', 'k', 'so4', 'cl', 'f', 'nh4_n',
       'no2_no3_n', 'po4_p', 'tot_p', 'si', 'fe', 'oxy', 'water_temp', 'tot_n',
       'abs420', 'toc_ton', 'toc_top', 'altitud', 'aro_area', 'forest',
       'water', 'marshland', 'Hygge', 'openground', 'farmland', 'mountain',
       'urban'],
      dtype='object')

In [9]:

# Read OSM data - get administrative boundaries

# define the place query
query = {'country': 'Sweden'}

# get the boundaries of the place (add additional buffer around the query)
boundaries = ox.geocode_to_gdf(query, buffer_dist=5000)

# Let's check the boundaries on a map
boundaries.explore()

C:\Users\vijay\AppData\Local\Temp\ipykernel_12652\2145721211.py:7: UserWarning: The buffer_dist argument has been deprecated and will be removed in a future release. Buffer your results directly, if desired.
  boundaries = ox.geocode_to_gdf(query, buffer_dist=5000)


In [10]:
# Create a GeoDataFrame
landdata["geometry"] = gpd.points_from_xy(landdata["long"], landdata["lat"])
landdata = gpd.GeoDataFrame(landdata, crs="epsg:4326")

# Filter data geographically
landdata = gpd.sjoin(landdata, boundaries[["geometry"]])
landdata = landdata.reset_index(drop=True)

# Check the first rows
landdata.head()

,md_mvm_id,lat,long,toc,year,month,day,season_type,season,max_sample_depth,...,forest,water,marshland,Hygge,openground,farmland,mountain,urban,geometry,index_right
0,54,60.638793,15.445276,4.2,2020,10,12,1,4,0.5,...,73.469462,10.837264,3.676818,12.016456,0.000000,0.000000,0.0,0.000000,POINT (15.44528 60.63879),0
1,55,59.805629,17.903998,13.2,2020,10,26,1,4,0.5,...,78.878348,3.402606,3.824474,2.837472,8.661184,2.395915,0.0,0.000000,POINT (17.90400 59.80563),0
2,56,60.027726,15.662726,8.9,2020,10,21,1,4,0.5,...,75.968555,10.497775,3.313430,6.915198,2.147880,0.236199,0.0,0.920962,POINT (15.66273 60.02773),0
3,57,60.162144,15.733446,10.3,2020,10,21,1,4,0.5,...,71.366119,13.571100,4.610804,9.689102,0.762875,0.000000,0.0,0.000000,POINT (15.73345 60.16214),0
4,58,60.645048,13.626457,15.9,2020,10,13,1,4,0.5,...,69.157181,8.154980,19.268154,2.410018,1.009667,0.000000,0.0,0.000000,POINT (13.62646 60.64505),0


In [12]:
# Here the tooltip parameter specifies which attributes are shown when hovering on top of the points
# The vmax parameter specifies the maximum value for the colormap (here, all 1000 dollars and above are combined)
landdata.explore(column="toc", cmap="Reds", scheme="quantiles", tooltip=["toc"], tiles="CartoDB positron")

In [11]:
explanatory_vars = ['ph', 'mg', 'fe', 'tot_p', 'water_temp', 'cpyll', 'oxy','si', 'cond25', 'nh4_n','toc_ton', 'toc_top', 'altitud']

In [13]:
# creating a lag variable based on the forest coverage in the landscape
def has_forest_cover(forest):
    if forest > 50.0:
        return 1
    else:
        return 0


landdata['forest_cover'] = landdata['forest'].apply(has_forest_cover)


In [15]:
landdata.head()

,md_mvm_id,lat,long,toc,year,month,day,season_type,season,max_sample_depth,...,water,marshland,Hygge,openground,farmland,mountain,urban,geometry,index_right,forest_cover
0,54,60.638793,15.445276,4.2,2020,10,12,1,4,0.5,...,10.837264,3.676818,12.016456,0.000000,0.000000,0.0,0.000000,POINT (15.44528 60.63879),0,1
1,55,59.805629,17.903998,13.2,2020,10,26,1,4,0.5,...,3.402606,3.824474,2.837472,8.661184,2.395915,0.0,0.000000,POINT (17.90400 59.80563),0,1
2,56,60.027726,15.662726,8.9,2020,10,21,1,4,0.5,...,10.497775,3.313430,6.915198,2.147880,0.236199,0.0,0.920962,POINT (15.66273 60.02773),0,1
3,57,60.162144,15.733446,10.3,2020,10,21,1,4,0.5,...,13.571100,4.610804,9.689102,0.762875,0.000000,0.0,0.000000,POINT (15.73345 60.16214),0,1
4,58,60.645048,13.626457,15.9,2020,10,13,1,4,0.5,...,8.154980,19.268154,2.410018,1.009667,0.000000,0.0,0.000000,POINT (13.62646 60.64505),0,1


In [16]:
all_model_attributes = ["toc"] + explanatory_vars
has_nans = False
for attr in all_model_attributes:
    if landdata[attr].hasnans:
        has_nans = True
print("Has missing values:", has_nans)

Has missing values: False


In [17]:
landdata = landdata.dropna(subset=all_model_attributes).copy()

In [18]:
w_forest = weights.KNN.from_shapefile('E:/test2/shapefile.shp', k=8)
# Assign spatially lag based on the forest cover in the landscape 
lagged = landdata.assign(w_forest=weights.spatial_lag.lag_spatial(w_forest, landdata['forest_cover'].values))
lagged.head()

,md_mvm_id,lat,long,toc,year,month,day,season_type,season,max_sample_depth,...,marshland,Hygge,openground,farmland,mountain,urban,geometry,index_right,forest_cover,w_forest
0,54,60.638793,15.445276,4.2,2020,10,12,1,4,0.5,...,3.676818,12.016456,0.000000,0.000000,0.0,0.000000,POINT (15.44528 60.63879),0,1,8.0
1,55,59.805629,17.903998,13.2,2020,10,26,1,4,0.5,...,3.824474,2.837472,8.661184,2.395915,0.0,0.000000,POINT (17.90400 59.80563),0,1,7.0
2,56,60.027726,15.662726,8.9,2020,10,21,1,4,0.5,...,3.313430,6.915198,2.147880,0.236199,0.0,0.920962,POINT (15.66273 60.02773),0,1,8.0
3,57,60.162144,15.733446,10.3,2020,10,21,1,4,0.5,...,4.610804,9.689102,0.762875,0.000000,0.0,0.000000,POINT (15.73345 60.16214),0,1,8.0
4,58,60.645048,13.626457,15.9,2020,10,13,1,4,0.5,...,19.268154,2.410018,1.009667,0.000000,0.0,0.000000,POINT (13.62646 60.64505),0,1,8.0


In [19]:
extended_vars = explanatory_vars + ["forest", "w_forest"]

m2 = spreg.OLS(lagged[['toc']].values, lagged[extended_vars].values, name_y = 'toc', name_x=extended_vars)

In [20]:
print(m2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :         toc                Number of Observations:         105
Mean dependent var  :     10.0629                Number of Variables   :          16
S.D. dependent var  :      6.1138                Degrees of Freedom    :          89
R-squared           :      0.8142
Adjusted R-squared  :      0.7829
Sum squared residual:     722.349                F-statistic           :     25.9970
Sigma-square        :       8.116                Prob(F-statistic)     :   3.778e-26
S.E. of regression  :       2.849                Log likelihood        :    -250.237
Sigma-square ML     :       6.880                Akaike info criterion :     532.475
S.E of regression ML:      2.6229                Schwarz criterion     :     574.938

-----------------------------------------------------------------------------

Spatial Lag Model Wy

In [23]:
variables = explanatory_vars + ["forest"]
m3 = spreg.GM_Lag(landdata[['toc']].values, landdata[variables].values,w=w,name_y = 'toc', name_x = variables)

In [24]:
print(m3.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :         toc                Number of Observations:         105
Mean dependent var  :     10.0629                Number of Variables   :          16
S.D. dependent var  :      6.1138                Degrees of Freedom    :          89
Pseudo R-squared    :      0.8081
Spatial Pseudo R-squared:  0.8083

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      17.6926213       5.4484008       3.2473054       0.0011650
                  ph      -2.7716775       0.5774205      -4.8001022       0.0000016
                  mg     -11.2503508       5.8613619      -

In [ ]:
from sklearn.metrics import mean_squared_error as mse

mses = pd.Series({'OLS': mse(landdata["toc"], m1.predy.flatten()), \
                  'OLS+W': mse(landdata["toc"], m2.predy.flatten()), \
                  'Lag': mse(landdata["toc"], m3.predy_e)
                    })
mses.sort_values()

OLS+W    6.879510
Lag      7.098275
OLS      7.131075
dtype: float64

In [ ]:
from sklearn.metrics import mean_absolute_error

mae = pd.Series({'OLS': mean_absolute_error(landdata["toc"], m1.predy.flatten()), \
                  'OLS+W': mean_absolute_error(landdata["toc"], m2.predy.flatten()), \
                  'Lag': mean_absolute_error(landdata["toc"], m3.predy_e)
                    })
mae.sort_values()

OLS+W    2.009845
Lag      2.026829
OLS      2.031220
dtype: float64